In [2]:
import sys, os, numpy as np, pandas as pd
sys.path.append('../../../02_codes/modules')
from seq_funcs import fasta_to_dict, get_codons, aln_to_df

ModuleNotFoundError: No module named 'seq_funcs'

In [3]:
path_to_xxx = '/Volumes/WDR_2TB_B_/clustalO_alns/230418/out_230417/clustalo_out/alns_cut_bad_matches/Dm_1756_vs_A0A6H2EDL7_p84.45_u50.87_0.35.faa'
path_to_yyy = '/Volumes/WDR_2TB_B_/clustalO_alns/230418/out_230417/clustalo_out/alns_cut_bad_matches/Dm_357_vs_P41572_p99.79_u0.0_0.0.faa'
s
verbose     = 1

NameError: name 's' is not defined

In [41]:
aln_dict = fasta_to_dict(path_to_yyy)

In [43]:
pair_up = list(zip(aln_dict['Dm_357'], aln_dict['P41572']))
len(pair_up)

481

In [44]:
count  = 0
blocks = []
block  = []
while count < len(pair_up):
    pair = pair_up[count]
    if not '-' in pair: 
        block.append(pair)
    else:
        if not block == []:
            blocks.append(block)
            block = []
    count+=1

if not block == []:
    blocks.append(block)

In [45]:
len(blocks)

1

In [46]:
for block in blocks: 
    print(len(block))

481


In [39]:
def get_aln_blocks(aln_path, gap_char = '-', verbose = 0): 

    # read aln as dict
    aln_dict = fasta_to_dict(path_to_yyy)
    
    
    # algorithm to get blocks
    

242